<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
from skopt.learning import ExtraTreesRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm.sklearn import LGBMRegressor
from ngboost import NGBRegressor
from ngboost.distns import Normal
np.random.seed(42)
rnd = np.random.RandomState(42)

In [ ]:
X,y = make_regression(1000,100, random_state=rnd)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=rnd)

In [29]:
%%time
erf = ExtraTreesRegressor(n_estimators=10, max_depth=3, random_state=rnd)
erf.fit(X_train, y_train)
erf_mean, erf_std = erf.predict(X_test, return_std=True)
mean_squared_error(erf_mean, y_test), erf_std.mean()

CPU times: user 28.7 ms, sys: 0 ns, total: 28.7 ms
Wall time: 27.2 ms


(9655.225545166513, 127.33751033083503)

In [30]:
%%time
lgb = LGBMRegressor(n_estimators=10,
                    max_depth=3,
                    verbose=-1,
                    learning_rate=.1,
                    force_col_wise=True,
                    device="cpu")
ngb = NGBRegressor(Dist=Normal, Base=lgb,n_estimators=100).fit(X_train,y_train)
preds=ngb.predict(X_test)
dist = ngb.pred_dist(X_test)
mean_squared_error(y_test,preds), dist.params["scale"].mean()

[iter 0] loss=6.3891 val_loss=0.0000 scale=2.0000 norm=232.7423
CPU times: user 33.4 s, sys: 1.68 s, total: 35.1 s
Wall time: 2.95 s


(5862.552134014378, 78.29087437939015)